# 7.2 Data Transformation（数据变换）

# 1 删除重复值



In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame方法duplicated返回的是一个boolean Series，表示一个row是否是重复的（根据前一行来判断）：

In [3]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

drop_duplicateds返回一个DataFrame，会删除重复的部分：

In [4]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [5]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [6]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated和drop_duplicated默认保留第一次观测到的数值组合。设置`keep='last'`能返回最后一个：

In [7]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


# 2 Transforming Data Using a Function or Mapping（用函数和映射来转换数据）

有时候我们可能希望做一些数据转换。比如下面一个例子，有不同种类的肉：

In [8]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设你想加一列，表明每种肉来源的动物是什么。我们可以写一个映射：

In [9]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

用于series的map方法接受一个函数，或是一个字典，包含着映射关系，但这里有一个小问题，有些肉是大写，有些是小写。因此，我们先用str.lower把所有的值变为小写:

In [10]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [11]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们也可以用一个函数解决上面的问题：

In [12]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使用map是一个很简便的方法，用于element-wise转换和其他一些数据清洗操作。

# 3 Replacing Values（替换值）

其实fillna是一个特殊换的替换操作。map可以用于修改一个object里的部分值，但是replace能提供一个更简单和更灵活的方法做到这点。下面是一个series：

In [13]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

这里-999可能是用来表示缺失值的标识符。用NA来替代的话，用replace，会产生一个新series（除非使用inplace=True）:

In [14]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果想要一次替换多个值，直接用一个list即可：

In [15]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

对于不同的值用不同的替换值，也是导入一个list：

In [16]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

参数也可以是一个dict：

In [17]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64



# 4 Renaming Axis Indexes（重命名Axis Indexes）



In [18]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [19]:
transform = lambda x: x[:4].upper()
transform

<function __main__.<lambda>>

In [20]:
data.index

Index(['Ohio', 'Colorado', 'New York'], dtype='object')

In [21]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [22]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [23]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [24]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'pekaboo'})

,one,two,pekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [25]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


# 5 Discretization and Binning（离散化和装箱）


连续型数据经常被离散化或分散成bins（分箱）来分析。假设你有一组数据，你想把人分到不同的年龄组里：

In [26]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

我们把这些分到四个bin里，19~25, 26~35, 36~60, >60。可以用pandas里的cut：

In [27]:
bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins)

cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [28]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [29]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [30]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [31]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

你也可以用一个list或数组给labels选项来设定bin的名字：

In [32]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [33]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [34]:
data = np.random.rand(20)

In [35]:
pd.cut(data, 4, precision=2)

[(0.27, 0.5], (0.031, 0.27], (0.27, 0.5], (0.5, 0.74], (0.031, 0.27], ..., (0.031, 0.27], (0.5, 0.74], (0.27, 0.5], (0.74, 0.97], (0.031, 0.27]]
Length: 20
Categories (4, interval[float64]): [(0.031, 0.27] < (0.27, 0.5] < (0.5, 0.74] < (0.74, 0.97]]

In [36]:
data = np.random.randn(1000) # Normally distributed

In [37]:
cats = pd.qcut(data, 4) # Cut into quartiles
cats

[(-2.911, -0.64], (-0.64, 0.0308], (-2.911, -0.64], (0.0308, 0.719], (-2.911, -0.64], ..., (-2.911, -0.64], (-0.64, 0.0308], (-2.911, -0.64], (-0.64, 0.0308], (-0.64, 0.0308]]
Length: 1000
Categories (4, interval[float64]): [(-2.911, -0.64] < (-0.64, 0.0308] < (0.0308, 0.719] < (0.719, 3.275]]

In [38]:
pd.value_counts(cats)

(0.719, 3.275]     250
(0.0308, 0.719]    250
(-0.64, 0.0308]    250
(-2.911, -0.64]    250
dtype: int64

类似的，在cut中我们可以自己指定百分比：

In [39]:
cats2 = pd.cut(data, [0, 0.1, 0.5, 0.9, 1.]) # 累进的百分比
cats2

[NaN, NaN, NaN, (0.5, 0.9], NaN, ..., NaN, NaN, NaN, NaN, NaN]
Length: 1000
Categories (4, interval[float64]): [(0.0, 0.1] < (0.1, 0.5] < (0.5, 0.9] < (0.9, 1.0]]

In [40]:
pd.value_counts(cats2)

(0.1, 0.5]    157
(0.5, 0.9]    129
(0.0, 0.1]     33
(0.9, 1.0]     27
dtype: int64

在之后的章节我们还会用到cut和qcut，这些离散函数对于量化和群聚分析很有用。

# 6 Detecting and Filtering Outliers（检测和过滤异常值）

过滤或转换异常值是数组操作的一个重头戏。下面的DataFrame有正态分布的数据：

In [41]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.012470,-0.038583,-0.021297,0.022002
std,0.993969,1.005739,0.955211,0.992542
min,-3.131648,-3.028107,-2.913405,-3.010661
25%,-0.699019,-0.699017,-0.702823,-0.648066
50%,-0.006753,-0.035163,-0.015562,-0.018169
75%,0.654827,0.662917,0.629019,0.717173
max,3.772542,3.224617,2.811125,3.451829


假设我们想要找一个列中，绝对值大于3的数字：

In [42]:
data.head()

,0,1,2,3
0,0.614727,0.439992,0.147880,1.880868
1,-1.047613,1.144173,1.708863,0.722797
2,1.508174,1.189739,0.258907,-1.095028
3,-0.742261,-0.730732,0.879664,-0.678631
4,1.319965,-1.740188,1.211800,0.497287


In [43]:
col = data[2]
col.head()

0    0.147880
1    1.708863
2    0.258907
3    0.879664
4    1.211800
Name: 2, dtype: float64

In [44]:
col[np.abs(col) > 3]

Series([], Name: 2, dtype: float64)

选中所有绝对值大于3的行，可以用any方法在一个boolean DataFrame上：

In [45]:
data[(np.abs(data) > 3)].head()

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [46]:
data[(np.abs(data) > 3).any(1)] # any中axis=1表示column

,0,1,2,3
48,-0.203092,-1.346949,-1.553816,-3.010661
63,-0.133207,0.676213,0.657501,3.047416
114,3.085339,1.171219,0.663044,-0.910114
140,-3.131648,0.888285,1.088128,0.715026
218,-0.734757,3.224617,0.794693,0.154402
342,3.011635,-0.617831,1.014915,-0.096441
355,3.151843,0.205694,-0.198880,-1.333455
817,0.930330,-0.337456,1.571384,3.059690
820,3.772542,-1.071938,-0.413481,0.989101
877,1.055055,-3.028107,0.336986,-0.559786


下面是把绝对值大于3的数字直接变成-3或3：

In [47]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [48]:
data[21:23]

,0,1,2,3
21,-0.150737,0.167932,-0.305328,-1.095246
22,-0.099395,-0.785022,0.017150,1.244723


In [49]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.013359,-0.038780,-0.021297,0.021454
std,0.990142,1.004951,0.955211,0.990723
min,-3.000000,-3.000000,-2.913405,-3.000000
25%,-0.699019,-0.699017,-0.702823,-0.648066
50%,-0.006753,-0.035163,-0.015562,-0.018169
75%,0.654827,0.662917,0.629019,0.717173
max,3.000000,3.000000,2.811125,3.000000


np.sign(data)会根据值的正负号来得到1或-1：

In [50]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,1.0,1.0
1,-1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,-1.0
3,-1.0,-1.0,1.0,-1.0
4,1.0,-1.0,1.0,1.0


# 7 Permutation and Random Sampling（排列和随机采样）

排列（随机排序）一个series或DataFrame中的row，用numpy.random.permutation函数很容易就能做到。调用permutation的时候设定好你想要进行排列的axis，会产生一个整数数组表示新的顺序：

In [51]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [52]:
sampler = np.random.permutation(5)
sampler

array([3, 2, 1, 0, 4])

这个数组能被用在基于iloc上的indexing或take函数：

In [53]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19


为了选中一个随机的子集，而且没有代替功能(既不影响原来的值，返回一个新的series或DataFrame)，可以用sample方法：

In [54]:
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
4,16,17,18,19
2,8,9,10,11


如果想要生成的样本带有替代功能(即允许重复)，给sample中设定replace=True:

In [55]:
choices = pd.Series([5, 7, -1, 6, 4])

draws = choices.sample(n=10, replace=True)

draws

3    6
0    5
1    7
4    4
0    5
4    4
4    4
4    4
0    5
2   -1
dtype: int64

# 8 Computing Indicator/Dummy Variables



In [56]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [57]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


在一些情况里，如果我们想要给column加一个prefix， 可以用data.get_dummies里的prefix参数来实现：

In [58]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [59]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [60]:
mnames = ['movie_id', 'title', 'genres']

In [61]:
movies = pd.read_table('../datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames, engine='python')
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [62]:
all_genres = []

for x in movies.genres:
    all_genres.extend(x.split('|'))
    
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [63]:
zero_matrix = np.zeros((len(movies), len(genres)))

zero_matrix.shape

(3883, 18)

In [64]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


然后迭代每一部movie，并设置每一行中的dummies为1。使用dummies.columns来计算每一列的genre的指示器：

In [65]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [66]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [67]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [68]:
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [70]:
np.random.seed(12345)

In [71]:
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [72]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.]

In [73]:
pd.cut(values, bins)

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [74]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0
